In [30]:
import pandas as pd
df = pd.read_csv('yahoo_finance_articles.csv',usecols=['content'])
df.head()
df_0 = df.copy()

In [39]:
#tokenization
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
import re

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s.]', '', text)
    text = ' '.join(text.split())
    return text

def tokenize_text(text):
    if pd.isna(text):
        return []
    clean_text = preprocess_text(text)
    tokens = word_tokenize(clean_text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens 
             if token not in stop_words 
             and token not in string.punctuation]    
    return tokens

df['content_tokens'] = df['content'].apply(lambda x: tokenize_text(x))
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hrithik/codes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hrithik/codes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,content,tokenize_content,content_tokens
0,In This Article: The Nasdaq led US stocks high...,"[In, This, Article, :, The, Nasdaq, led, US, s...","[article, nasdaq, led, us, stocks, higher, wed..."
1,In This Article: Netflix stock (NFLX) surged t...,"[In, This, Article, :, Netflix, stock, (, NFLX...","[article, netflix, stock, nflx, surged, alltim..."
2,In This Article: Nvidia (NVDA) stock jumped mo...,"[In, This, Article, :, Nvidia, (, NVDA, ), sto...","[article, nvidia, nvda, stock, jumped, 4, wedn..."
3,In This Article: JPMorgan Chase (JPM) CEO Jami...,"[In, This, Article, :, JPMorgan, Chase, (, JPM...","[article, jpmorgan, chase, jpm, ceo, jamie, di..."
4,In This Article: By Stephen Nellis and Krystal...,"[In, This, Article, :, By, Stephen, Nellis, an...","[article, stephen, nellis, krystal, hu, reuter..."
